In [ ]:
import datetime
import femr
import pandas as pd
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from sklearn.model_selection import train_test_split
import os
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import logging
import pickle
import re
from scipy.stats import spearmanr
from scipy.stats import zscore
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset



# Create Foundation Model Representations

In [ ]:
import os
import femr
import femr.etl_pipelines.simple
import shutil

INPUT_DIR = 'input/omop'
TARGET_DIR = 'trash/extracts'
LOG_DIR = os.path.join(TARGET_DIR, "logs")
EXTRACT_DIR = os.path.join(TARGET_DIR, "extract")
if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

os.mkdir(TARGET_DIR)

# Run extract generation
os.system(f"etl_generic_omop {INPUT_DIR} {EXTRACT_DIR} {LOG_DIR} --num_threads 2")

In [ ]:
# Create labels (a dataframe in specified format of FEMRv1)
## for every patient, take the DOS (days to onset of labor) number and subtract it from corresponding child_birth_date of liked EHR
labeldf['child_birth_date'] = pd.to_datetime(labeldf['child_birth_date'])
labeldf['protein_date'] = labeldf['child_birth_date'] + pd.to_timedelta(labeldf['DOS'], unit='D')
labeldf['protein_date'] = pd.to_datetime(labeldf['protein_date']).dt.strftime('%Y-%m-%dT%H:%M:%S')
labeldf

In [ ]:
%%time
MODEL = '.path_to_foundation_model/'
EXTRACT_LOCATION = './trash/extracts/extract'
LABELS = './trash/labels/labeldf.csv'

# Compute Foundation Model Representation
os.system(
    f"femr_compute_representations --data_path {EXTRACT_LOCATION} --model_path {MODEL} --prediction_times_path {LABELS} --batch_size 1024 ./trash/motor_rep/motor_reprs.pkl"
)

In [ ]:
%%time
"""
Open the resulting representations and take a look at the data matrix.
"""
import pickle

REPRESENTATIONS = './trash/motor_rep/motor_reprs.pkl'

with open(REPRESENTATIONS, "rb") as f:
    reprs = pickle.load(f)

    print(reprs.keys())

    print("Pulling the data for the first label")
    print("Patient id", reprs["patient_ids"][1])
    print("Label time", reprs["prediction_times"][1])
    print("Representation", reprs["representations"][1, :16])

In [ ]:
# Convert representations to dataframe for easier manipulation
df_initial = pd.DataFrame({
    'patient_ids': reprs['patient_ids'],
    'labeling_time': reprs['prediction_times']
})

# Convert 'data_matrix' into a DataFrame
df_data_matrix = pd.DataFrame(reprs['representations'], columns=[f'data_{i}' for i in range(769)])

# Concatenate the two DataFrames along the columns axis
df_reprs = pd.concat([df_initial, df_data_matrix], axis=1)

# Column data_768 marks boolean labels (not used here) so drop
df_reprs = df_reprs.drop(columns='data_768')
df_reprs.head()

In [ ]:
# Process dataframe to specific format for easier downstream processing
# Convert 'protein_date' to the same format as 'labeling_time'
labeldf['protein_date'] = pd.to_datetime(labeldf['protein_date']).dt.strftime('%Y-%m-%d')
# Convert both 'labeling_time' and 'protein_date' to datetime format
df_reprs['labeling_time'] = pd.to_datetime(df_reprs['labeling_time'])
labeldf['protein_date'] = pd.to_datetime(labeldf['protein_date'])
# Convert 'cln_sample_ID' to the same data type as 'patient_ids'
labeldf['cln_sample_ID'] = labeldf['cln_sample_ID'].astype(str)
df_reprs['patient_ids'] = df_reprs['patient_ids'].astype(str)

# Merge the dataframes again to get the corresponding sample_ID
df_merged = pd.merge(df_reprs, labeldf, left_on=['patient_ids', 'labeling_time'], right_on=['cln_sample_ID', 'protein_date'], how='left')

# Keep all columns from df_reprs and just add the sample_ID column
df_reprs_final = df_merged[['sample_ID'] + df_reprs.columns.tolist()]

df_reprs_final


# Training pipeline for proteomics generation

In [ ]:
# Set empty dataframe to collect stats
data = {"protein": [], "spearman_correlation": [], "spearman_pvalue": [], "best_train_rmse": [],  "best_val_rmse": []}
df = pd.DataFrame(data)


In [ ]:
# Set up empty dataframe to collect prediction values
predvalues_df = pd.DataFrame({'sample_index': range(171)})
# Function to populate the dataframe with new columns
def populate_dataframe(df, poi, predictions):
    column_name = poi + "_protein"
    df[column_name] = predictions
    return df

In [ ]:
# Read representation and proteomics data
df_reprs_final= pd.read_csv('.path_to_FM_representation/motor_pregnancy_cohort_reprs.csv')
prot = pd.read_csv('.path_to_proteomics_data/proteomics_outcomes.csv')

def data_org(protein_name, latentdf, prot):

    # Protein of interest
    poi = protein_name
    poiname = poi + '_protein'
    print(poiname)
    
    # Subset protein expression for poi
    poidf = prot[[poiname, 'sample_ID']]
    # Merge dfs and clean for xgboost by removing sample_ID and extraneous columns
    merged_df = pd.merge(latentdf, poidf, on='sample_ID')
    cols_remove = ['sample_ID', 'labeling_time'] ## for bypatient split
    merged_clean = merged_df.drop(columns=cols_remove)

    # Create x and y dataframes
    X, y = merged_clean.drop(poiname, axis=1), merged_clean[[poiname, 'patient_ids']]
    return poi, X, y


def bootstrap_nn(poi, X, y, bootstrap):    
    # Create data structures below for boostrapping
    train_error_dict = {}
    val_error_dict = {}
    prediction_dict = {str(i): None for i in range(y.shape[0])}
    poiname = poi + '_protein'
    
    unique_patient_ids = X['patient_ids'].unique()## for bypatient split
    unique_patient_ids_df = pd.DataFrame(unique_patient_ids, columns=['patient_ids']) ## for bypatient split
    
    for i in range(bootstrap):
        # Split patient IDs
        unique_patient_ids = X['patient_ids'].unique()
        unique_patient_ids_df = pd.DataFrame(unique_patient_ids, columns=['patient_ids'])
        
        # Assuming one column ('patient_ids') is not a feature
        input_size = X.shape[1] - 1
        # Split data 50/50 into train/val_and_test
        train_ids_initial, test_ids = train_test_split(unique_patient_ids_df, test_size=0.5, random_state=i)
        
        # Split data into initial train and test sets
        cols_remove = ['patient_ids']
        X_train_initial = X[X['patient_ids'].isin(train_ids_initial['patient_ids'])]
        X_test = X[X['patient_ids'].isin(test_ids['patient_ids'])].drop(columns=cols_remove)
        y_train_initial = y[y['patient_ids'].isin(train_ids_initial['patient_ids'])]
        y_test = y[y['patient_ids'].isin(test_ids['patient_ids'])].drop(columns=cols_remove)
        
        # Further split the initial training data into training and validation sets (80/20 split)
        train_ids, val_ids = train_test_split(train_ids_initial, test_size=0.2, random_state=i)
        
        X_train = X_train_initial[X_train_initial['patient_ids'].isin(train_ids['patient_ids'])].drop(columns=cols_remove)
        X_val = X_train_initial[X_train_initial['patient_ids'].isin(val_ids['patient_ids'])].drop(columns=cols_remove)
        y_train = y_train_initial[y_train_initial['patient_ids'].isin(train_ids['patient_ids'])].drop(columns=cols_remove)
        y_val = y_train_initial[y_train_initial['patient_ids'].isin(val_ids['patient_ids'])].drop(columns=cols_remove)

        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
        
        # Create datasets and dataloaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        
        # Build Model
        class RegressionModel(nn.Module):
            def __init__(self):
                super(RegressionModel, self).__init__()
                self.layer1 = nn.Linear(input_size, 32)  # Input to hidden layer
                self.relu = nn.ReLU()
                self.layer2 = nn.Linear(32, 1)  # Hidden to output layer
        
            def forward(self, x):
                x = self.relu(self.layer1(x))
                x = self.layer2(x)
                return x
        
        model = RegressionModel()
        
        # Criterion and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Training loop
        num_epochs = 1000
        train_losses = []
        val_losses = []
        
        best_val_loss = float('inf')
        best_model_state = None
        best_epoch = 0
        
        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item() * inputs.size(0)
            train_loss = train_loss / len(train_loader.dataset)
            train_losses.append(train_loss)
            
            # Validation phase
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
            val_loss = val_loss / len(val_loader.dataset)
            val_losses.append(val_loss)
            
            # Update best model if current validation loss is lower
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict()
                best_epoch = epoch + 1
        
            # Print training and validation loss for each epoch
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        
        # Store training and validation losses for plotting
        train_error_dict[i] = train_losses
        val_error_dict[i] = val_losses
        
        # Print best epoch
        print(f'The best model was from epoch {best_epoch} with Validation Loss: {best_val_loss:.4f}')
        
        # Load best model state
        model.load_state_dict(best_model_state)
        
        # Collect actual and predicted values for the best model
        actual = []
        predicted = []
        
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                actual.extend(labels.numpy().flatten())
                predicted.extend(outputs.numpy().flatten())


        # Append predictions to prediction dictionary
        for p in range(y_test.shape[0]):
            key_to_append = str(y_test.index[p])
            value_to_append = predicted[p]
            if key_to_append in prediction_dict:
                if prediction_dict[key_to_append] is None:
                    prediction_dict[key_to_append] = []
                prediction_dict[key_to_append].append(value_to_append)
                

    return prediction_dict, train_error_dict, val_error_dict



def plot_correlation(poi, y, prediction_dict, savepath):
    from scipy.stats import pearsonr, spearmanr
    # Calculate average prediction for each sample
    y_avg_pred = [sum(values) / len(values) if values is not None else None for values in prediction_dict.values()]
    
    # Convert y_test DataFrame to a numpy array and reshape
    cols_remove = ['patient_ids']## for bypatient split
    y = y.drop(columns=cols_remove) ## for bypatient split
    
    y_np = y.to_numpy().reshape(-1)

    # Calculate correlation between predicted and actual values
    scorrelation, spvalue = spearmanr(y_np, y_avg_pred)
    
    # Plot predicted vs. actual values
    plt.scatter(y_np, y_avg_pred, alpha=0.5)
    plt.xlabel("Actual Values")
    plt.ylabel("Average Predicted Values")
    plt.title(f"{poi} Predicted vs. Actual\n(Spearman Correlation: {scorrelation:.2f}, Pvalue: {spvalue})")
    
    min_val = min(min(y_np), min(y_avg_pred))
    max_val = max(max(y_np), max(y_avg_pred))
    plt.xlim(min_val, max_val)
    plt.ylim(min_val, max_val)
    # Drawing a 45-degree line
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')
    # Save the plot as an image with the file name "poi.png"
    save_directory = savepath+"/correlation/"
    modpoi = poi.replace("/", "_")
    plot_filename = os.path.join(save_directory, f"{modpoi}_scorr.png")
    plt.savefig(plot_filename)

    plt.clf()
    return poi, scorrelation, spvalue, y_avg_pred


def plot_learning_curve(poi, train_error_dict, val_error_dict, savepath):
    # Calculate the average training error per epoch across all bootstrap iterations
    # Convert the dictionary values to a numpy array for easier calculation
    train_errors = np.array(list(train_error_dict.values()))
    val_errors = np.array(list(val_error_dict.values()))

    # Calculate the mean error per epoch across all bootstrap iterations
    mean_train_errors = np.mean(train_errors, axis=0)
    mean_val_errors = np.mean(val_errors, axis=0)

    # Epochs x-axis
    epochs = np.arange(1, len(mean_train_errors) + 1)
    
   # Plot the learning curve
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, mean_train_errors, label='Average Train Error', marker='o')
    plt.plot(epochs, mean_val_errors, label='Average Validation Error', marker='x')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.title(f'{poi} Learning Curve')
    plt.legend()
    plt.grid(True)
    
    # Save the plot
    save_directory = os.path.join(savepath, "learning_curves")
    os.makedirs(save_directory, exist_ok=True)
    plot_filename = os.path.join(save_directory, f"{poi.replace('/', '_')}_learning_curve.png")
    plt.savefig(plot_filename)
    
    plt.clf()  # Clear the figure to free memory
    
    # Return the minimum average training error
    best_train_error = np.min(mean_train_errors)
    best_train_rmse = np.sqrt(best_train_error)
    best_val_error = np.min(mean_val_errors)
    best_val_rmse = np.sqrt(best_val_error)
    return best_train_rmse, best_val_rmse
    



In [ ]:
# Iterate through proteins
outputdir = ".path_to_output_directory/motor_prot_pred"
for i in range(len(poi_list)):
    print("i index:", i)
    start_time = time.time()
    poi = poi_list[i]
    bootstrap = 10
    
    poi, X, y = data_org(poi, df_reprs_final, prot)

    predictions, train_error_dict, valid_error_dict =  bootstrap_nn(poi, X, y, bootstrap)
    poi, scorrelation, spvalue, y_avg_pred = plot_correlation(poi, y, predictions, outputdir)
    best_train_rmse_error, best_val_rmse_error = plot_learning_curve(poi, train_error_dict, valid_error_dict, outputdir)

    predvalues_df = populate_dataframe(predvalues_df, poi, y_avg_pred)
    predvalues_df.to_csv(outputdir+'/predvalues.csv', index=False)

    df.loc[i] = [poi, scorrelation, spvalue, best_train_rmse_error, best_val_rmse_error]
    df.to_csv(outputdir+'/output.csv', index=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time spent on grid search: {elapsed_time:.2f} seconds")


In [ ]:
# Adjust p-values
def benjamini_hochberg(p_values, alpha=0.05):
    m = len(p_values)
    sorted_indices = p_values.argsort()
    sorted_p_values = p_values[sorted_indices]

    adjusted_p_values = sorted_p_values.copy()
    for i in reversed(range(m)):
        if i == m - 1:
            adjusted_p_values[i] = min(sorted_p_values[i], 1)
        else:
            adjusted_p_values[i] = min(sorted_p_values[i] * m / (i + 1), adjusted_p_values[i + 1])

    # Reordering to the original order of p-values
    back_order_indices = sorted_indices.argsort()
    return adjusted_p_values[back_order_indices]

# Calculate BH corrected p-values for spearman_pvalues
output['bh_corrected_spearman_pvalue'] = benjamini_hochberg(output['spearman_pvalue'].values)

output.to_csv(outputdir+'/output_adjusted.csv', index=False)


In [ ]:
# add sample ID to predvalues csv (optional)
def data_org(protein_name, latentdf, prot):

     #protein of interest
    poi = protein_name #change this to select different proteins
    poiname = poi + '_protein'
    print(poiname)
    #subset protein expression for poi
    poidf = prot[[poiname, 'sample_ID']]
    #merge dfs and clean for xgboost by removing sample_ID and extraneous columns
    merged_df = pd.merge(latentdf, poidf, on='sample_ID')
    # Create x and y dataframes
    X, y = merged_df.drop(poiname, axis=1), merged_df[[poiname, 'sample_ID']]
    return poi, X, y


# To get the sample indexes in the correct order as the predicted values dataframe
poi, X, y = data_org('ROR1', df_reprs_final, prot)

predvalues_df.insert(1, 'sample_ID', y['sample_ID'])
predvalues_df.to_csv(outputdir+ '/predvalues.csv', index=False)